In [101]:
import pandas as pd
from textblob import TextBlob
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from collections import Counter
import string


nltk.download('punkt')
nltk.download('stopwords')


df = pd.read_csv("orders.csv")

[nltk_data] Error loading punkt: <urlopen error [SSL:
[nltk_data]     CERTIFICATE_VERIFY_FAILED] certificate verify failed:
[nltk_data]     unable to get local issuer certificate (_ssl.c:1000)>
[nltk_data] Error loading stopwords: <urlopen error [SSL:
[nltk_data]     CERTIFICATE_VERIFY_FAILED] certificate verify failed:
[nltk_data]     unable to get local issuer certificate (_ssl.c:1000)>


In [102]:
df['review_length'] = df['review_text'].apply(lambda x: len(x.split()))
df['sentiment_polarity'] = df['review_text'].apply(lambda x: TextBlob(x).sentiment.polarity)
df['sentiment_subjectivity'] = df['review_text'].apply(lambda x: TextBlob(x).sentiment.subjectivity)


In [103]:
def clean_text(text):
    return text.translate(str.maketrans('', '', string.punctuation)).lower()

df['clean_text'] = df['review_text'].apply(clean_text)

In [104]:
negative_keywords = [
    'late', 'delay', 'delayed', 'slow', 'still waiting', 'took too long', 'not on time',
   
    'damaged', 'broken', 'crushed', 'torn', 'opened', 'leaking', 'wet',
    
    'lost', 'missing', 'not delivered', 'never arrived', 'never got it', 
    "didn't receive", 'no package', 'nowhere to be found', 'disappeared', 'vanished',
    
    'wrong address', 'left outside', 'left unattended', 'not my address', 'misdelivered',

    'bad service', 'rude', 'unhelpful', 'no update', 'tracking issue', 
    'tracking didn’t work', 'no tracking info', 'unresponsive',

    'bad', 'terrible', 'awful', 'poor', 'worst', 'disappointed', 'inconvenient'
]
def has_negative_keyword(text):
    return int(any(word in text for word in negative_keywords))

df['has_negative_keyword'] = df['clean_text'].apply(has_negative_keyword)


In [105]:
keywords_dict = {
    'late': ['late', 'delayed', 'delay', 'took too long', 'not on time'],
    'lost': ['lost', 'missing', 'not delivered', 'never arrived', 'never got it', "didn't receive"],
    'broken': ['damaged', 'broken', 'crushed', 'torn', 'opened', 'leaking'],
    'address': ['wrong address', 'misdelivered', 'left outside', 'not my address'],
}
def keyword_group_count(text, keywords):
    text = text.lower()
    return sum(text.count(word) for word in keywords)

for label, keyword_list in keywords_dict.items():
    df[f'{label}_count'] = df['review_text'].apply(lambda x: keyword_group_count(x, keyword_list))

In [106]:
df[['review_text', 'review_length', 'sentiment_polarity', 
    'late_count', 'lost_count', 'broken_count','address_count']].head()


,review_text,review_length,sentiment_polarity,late_count,lost_count,broken_count,address_count
0,"Arrived on time, well packed.",5,0.0000,0,0,0,0
1,"Delivered late, box was damaged.",5,-0.3000,1,0,1,0
2,Smooth delivery.,2,0.4000,0,0,0,0
3,Tracking was inaccurate.,3,0.0000,0,0,0,0
4,"Wrong address at first, fixed later.",6,-0.0375,1,0,0,1
